In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from pyarabic.araby import strip_tashkeel
from pyarabic.araby import normalize_ligature
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import keras
from keras import regularizers
import re
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
d = pd.read_excel('/content/Dialects.xlsx')
data = d.sample(frac=1).reset_index(drop=True)
text = []
for a_ in data['text']:
    a_0 = strip_tashkeel(a_)
    a_1 = normalize_ligature(a_0)
    a_2 = a_1.replace('،', '')
    a_3 = a_2.replace('؟', '')
    a_4 = a_3.replace('.', '')
    a_5 = a_4.replace('"', '')
    a = a_5.replace('-', '')
    text.append(a)
le = LabelEncoder()
dialect = le.fit_transform(data['dialect'])
data_0 = pd.DataFrame(list(zip(text, dialect)), columns =['text', 'dialect'])
data_0.dialect.value_counts()

3    38000
1    28000
2    24000
0    20000
Name: dialect, dtype: int64

In [39]:
data_0['l'] = data_0['text'].apply(lambda x: len(str(x).split(' ')))
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(data_0['text'].values)
X = tokenizer.texts_to_sequences(data_0['text'].values)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
x = pad_sequences(X, 50)
y = pd.get_dummies(data_0['dialect']).values

In [41]:
a_1 = []
a_2 = []
a_3 = []
a_4 = []
a_5 = []
kf = KFold(n_splits=10)
embedding_dim = 256
MAX_SEQUENCE_LENGTH = 50
for train_idx, test_idx in kf.split(x, y):
    x_train = x[train_idx]
    x_test = x[test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]
    model = Sequential()
    model.add(Embedding(nb_words, embedding_dim, input_length=MAX_SEQUENCE_LENGTH, trainable=True))
    model.add(GRU(units=256, return_sequences=True, activation='relu'))
    model.add(Dropout(0.2))
    model.add(GRU(units=128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation='softmax'))
    def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))
    model.compile(loss='categorical_crossentropy', optimizer= "adam", metrics=['accuracy',f1_m,precision_m, recall_m])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
    mc = ModelCheckpoint('Dialects_Model_1', monitor='val_loss', mode='min', verbose=1)
    history = model.fit(x_train, y_train, callbacks = [es,mc], batch_size = 1024, epochs=100, validation_split=0.5)
    Y_pred = model.predict(x_test)
    Y_pred_classes = np.argmax(Y_pred,axis = 1)
    Y_true = np.argmax(y_test,axis = 1)
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
    print(confusion_mtx)
    loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
    a_1.append(accuracy)
    print('accuracy', accuracy)
    a_2.append(loss)
    print('loss', loss)
    a_3.append(f1_score)
    print('f1_score', f1_score)
    a_4.append(precision)
    print('Precision', precision)
    a_5.append(recall)
    print('Recall', recall)
    print("\n")
ac = np.mean(a_1)
ac_ = np.std(a_1)
lo = np.mean(a_2)
lo_ = np.std(a_2)
fm = np.mean(a_3)
fm_ = np.std(a_3)
pr = np.mean(a_4)
pr_ = np.std(a_4)
re = np.mean(a_5)
re_ = np.std(a_5)
print("\n\n\n")
print("Simple RNN Classifier:")
print("Accuracy= ", "%.4f%% (+/- %.4f%%)" % (ac, ac_))
print("Loss= ", "%.4f%% (+/- %.4f%%)" % (lo, lo_))
print("F-measure= ", "%.4f%% (+/- %.4f%%)" % (fm, fm_))
print("Precision= ", "%.4f%% (+/- %.4f%%)" % (pr, pr_))
print("Recall= ", "%.4f%% (+/- %.4f%%)" % (re, re_))

Epoch 1/100
49/49 [==============================] - 25s 468ms/step - loss: 1.5514 - accuracy: 0.3760 - f1_m: 0.0951 - precision_m: 0.3370 - recall_m: 0.0591 - val_loss: 0.9374 - val_accuracy: 0.6237 - val_f1_m: 0.4218 - val_precision_m: 0.9098 - val_recall_m: 0.2747

Epoch 00001: saving model to Dialects_Model_1
INFO:tensorflow:Assets written to: Dialects_Model_1/assets
Epoch 2/100
49/49 [==============================] - 23s 470ms/step - loss: 0.6945 - accuracy: 0.7385 - f1_m: 0.6525 - precision_m: 0.9054 - recall_m: 0.5329 - val_loss: 0.4176 - val_accuracy: 0.8434 - val_f1_m: 0.8450 - val_precision_m: 0.8970 - val_recall_m: 0.7987

Epoch 00002: saving model to Dialects_Model_1
INFO:tensorflow:Assets written to: Dialects_Model_1/assets
Epoch 3/100
49/49 [==============================] - 23s 462ms/step - loss: 0.2332 - accuracy: 0.9200 - f1_m: 0.9198 - precision_m: 0.9458 - recall_m: 0.8953 - val_loss: 0.4047 - val_accuracy: 0.8474 - val_f1_m: 0.8517 - val_precision_m: 0.8833 - val_r